# Sentinel-2 Super-Resolution (4×)

**Upscale Sentinel-2 imagery from 10m/pixel → 2.5m/pixel using SwinIR**

Make sure GPU is enabled: `Runtime → Change runtime type → GPU`

In [ ]:
# Clone repository
!git clone https://github.com/Rishikarnatakam/Klymo.git
%cd Klymo

# Install dependencies
!pip install -q -r requirements.txt

In [ ]:
# Verify GPU
import torch
print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Google Earth Engine authentication (Colab method)
import ee

# Use Colab's built-in auth
from google.colab import auth
auth.authenticate_user()

# Initialize with high-volume endpoint for Colab
ee.Initialize(project='ee-project')  # This will work without a project ID in Colab

# Alternative: if above fails, try this:
# ee.Authenticate()
# ee.Initialize()

print("✓ GEE authenticated")

In [ ]:
# Kaggle authentication (for WorldStrat dataset)
from google.colab import files
print("Upload kaggle.json (from kaggle.com/settings):")
files.upload()
!mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download WorldStrat dataset (~2GB)
!kaggle datasets download -d julienco/worldstrat -p datasets/worldstrat --unzip
print("✓ Dataset downloaded")

In [ ]:
# Train SwinIR (3 epochs, ~15 min on GPU)
from src.training.finetune import finetune_swinir
from src.data.worldstrat_loader import WorldStratDataset
from pathlib import Path

train_ds = WorldStratDataset(root_dir=Path('datasets/worldstrat'), split='train', max_samples=200)
val_ds = WorldStratDataset(root_dir=Path('datasets/worldstrat'), split='validation', max_samples=40)
print(f"Train: {len(train_ds)}, Val: {len(val_ds)}")

results = finetune_swinir(
    train_dataset=train_ds,
    val_dataset=val_ds,
    batch_size=4,
    max_epochs=3,
    device=DEVICE,
)
print(f"✓ Training complete! Best loss: {results['best_loss']:.4f}")

In [ ]:
# Fetch real Sentinel-2 tile from Delhi
from src.data.gee_fetcher import GEEFetcher

# Skip re-auth since we already authenticated above
fetcher = GEEFetcher(authenticate=False)
fetcher.authenticated = True
import ee
fetcher.ee = ee

delhi_tile = fetcher.fetch_tile('delhi', tile_size=256)
print(f"Fetched tile: {delhi_tile.shape}")

In [ ]:
# Run super-resolution
from src.inference.pipeline import SuperResolutionPipeline

pipeline = SuperResolutionPipeline(device=DEVICE)
results = pipeline.run(delhi_tile)

print(f"Input:  {results['lr'].shape} (10m/pixel)")
print(f"Output: {results['sr'].shape} (2.5m/pixel)")

In [ ]:
# Visualize comparison
import matplotlib.pyplot as plt
from src.data.preprocessing import to_8bit_visualization
from skimage.transform import resize

lr = results['lr']
bicubic = results['bicubic']
sr = results['sr']

lr_up = resize(lr, sr.shape[:2], order=0, preserve_range=True)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
axes[0].imshow(to_8bit_visualization(lr_up))
axes[0].set_title('LR Input (10m/pixel)', fontsize=14)
axes[0].axis('off')

axes[1].imshow(to_8bit_visualization(bicubic))
axes[1].set_title('Bicubic 4× (Baseline)', fontsize=14)
axes[1].axis('off')

axes[2].imshow(to_8bit_visualization(sr))
axes[2].set_title('SwinIR 4× (2.5m/pixel)', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.savefig('outputs/visualizations/comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved to outputs/visualizations/comparison.png")

In [ ]:
# Compute metrics (PSNR, SSIM)
from src.metrics.psnr import compute_psnr
from src.metrics.ssim import compute_ssim

# Compare SR vs Bicubic (lower is worse baseline)
psnr_bicubic = compute_psnr(bicubic, sr)
ssim_bicubic = compute_ssim(bicubic, sr)

print(f"Metrics (SwinIR vs Bicubic):")
print(f"  PSNR: {psnr_bicubic:.2f} dB")
print(f"  SSIM: {ssim_bicubic:.4f}")

---
## 🎨 Interactive Streamlit UI (Optional)

Run the Streamlit app for an interactive before/after comparison slider.

In [ ]:
# Install localtunnel for exposing Streamlit
!npm install -g localtunnel

In [ ]:
# Run Streamlit with tunnel
import subprocess
import time

# Start Streamlit in background
streamlit_process = subprocess.Popen(
    ['streamlit', 'run', 'app/streamlit_app.py', '--server.port', '8501'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

time.sleep(5)  # Wait for Streamlit to start

# Get tunnel URL
!npx localtunnel --port 8501

In [ ]:
# Download results
from google.colab import files
files.download('outputs/visualizations/comparison.png')